## 케라스 모델과 레이어를 맞춤형으로 사용하기
##### url: https://www.tensorflow.org/guide/keras/custom_layers_and_models

- 목차
    - 설정
    - Layer 클래스
        - 레이어에서 가중치와 연산 캡슐화하기
        - 모범 예제: 입력 차원이 알려지기 전까지 가중치 생성 지연시키기
        - 재귀적으로 레이어 구성하기
        - 데이터가 모델을 통과하는 동안 생성되는 손실 값을 재귀적으로 수집하는 레이어
        - 선택적으로 레이어 직렬화 활성화하기
        - call 메소드의 training 인자가 가지는 특권
    - 모델 작성하기
        - Model 클래스
        - 하나로 합치기: 엔드 투 엔드 예제
        - 객체 지향 개발을 넘어서: 함수형 API
        
### 설정

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf



tf.keras.backend.clear_session() # 노트북의 상태를 쉽게 초기화할 수 있습니다.

## Layer 클래스
### 레이어에서 가중치와 연산 캡슐화하기
사용하게 될 주요 데이터구조는 `Layer` 입니다. 레이어는 상태(레이어의 가중치,'weight')와 입력을 출력으로 변환하는 것(호출,'call', 레이어의 순전파)을 캡슐화합니다.<br><br>
여기 밀집되어 연결된 레이어가 있습니다. 이것들은 변수 `w`와 `b`로 나타내어지는 상태를 가지고 있습니다. 

In [2]:
from tensorflow.keras import layers

class Linear(layers.Layer):
    
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(initial_value=w_init(shape=(input_dim, units),
                                                  dtype='float32'),
                             trainable=True)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(initial_value=b_init(shape=(units,),
                                                  dtype='float32'),
                             trainable=True)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

x = tf.ones((2, 2))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)

tf.Tensor(
[[-0.02640932  0.01592169 -0.05877681 -0.0904264 ]
 [-0.02640932  0.01592169 -0.05877681 -0.0904264 ]], shape=(2, 4), dtype=float32)


`w`와 `b`가 자동적으로 레이어 속성 집합으로써 추적된다는 것을 확인해보세요.

In [3]:
assert linear_layer.weights == [linear_layer.w, linear_layer.b]

또한, `add_weight` 메소드를 사용하여 레이어가 가진 가중치에 더 빠르게 접근할 수 있습니다.

In [4]:
class Linear(layers.Layer):
    
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(shape=(input_dim, units),
                                initializer='random_normal',
                                trainable=True)
        self.b = self.add_weight(shape=(units,),
                                initializer='zeros',
                                trainable=True)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
s = tf.ones((2, 2))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)

tf.Tensor(
[[ 0.08907721 -0.00604777  0.08206306 -0.02428109]
 [ 0.08907721 -0.00604777  0.08206306 -0.02428109]], shape=(2, 4), dtype=float32)


#### 레이어는 학습할 수 없는 가중치를 가질 수 있습니다. 
학습 가능한 가중치들 사이에서, 레이어에 학습할 수 없는 가중치들을 추가할 수 있습니다.<br>
레이어를 학습하는 과정에서 역전파 시 이러한 가중치들은 고려하지 않아야 합니다.<br><br>
어떻게 학습할 수 없는 가중치를 더하고 사용하는지 살펴보겠습니다.

In [5]:
class ComputeSum(layers.Layer):
    
    def __init__(self, input_dim):
        super(ComputeSum, self).__init__()
        self.total = tf.Variable(initial_value=tf.zeros((input_dim,)),
                                 trainable=False)
        
    def call(self, inputs):
        self.total.assign_add(tf.reduce_sum(inputs, axis=0))
        return self.total
    
x = tf.ones((2, 2))
my_sum = ComputeSum(2)
y = my_sum(x)
print(y.numpy())
y = my_sum(x)
print(y.numpy())

[2. 2.]
[4. 4.]


해당 가중치들은 `layer.weights`의 일부분이지만, 학습할 수 없는 가중치로 분류됩니다.

In [6]:
print('weights:', len(my_sum.weights))
print('non-trainable weights:', len(my_sum.non_trainable_weights))

# 학습할 수 있는 가중치가 포함되어 있지 않습니다.
print('trainable_weights:', my_sum.trainable_weights)

weights: 1
non-trainable weights: 1
trainable_weights: []


### 모범 예제: 입력 차원이 알려지기 전까지 가중치 생성 지연시키기

로지스틱 회귀 예시에서, `Linear` 레이어는 `__init__` 메소드에서 가중치 `w`와 `b`의 크기를 계산하기 위해 사용되는 `input_dim`을 인자로 가지고 있습니다.

In [7]:
class Linear(layers. Layer):
    
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(shape=(input_dim, units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(units, ),
                                 initializer='zeros',
                                 trainable=True)

많은 경우에서 입력의 크기를 미리 알 수 없습니다. 또한 레이어를 먼저 생성한 후, 입력 값이 알려졌을 때 게으르게(lazily) 가중치를 생성하길 원할 수도 있습니다.<br><br>
Keras API에서, 레이어의 가중치를 `build(input_shape)` 메소드에서 생성하는 것을 권합니다. 해당 메소드는 다음과 같습니다. 

In [8]:
class Linear(layers.Layer):
    
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units
        
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units, ),
                                 initializer='random_normal',
                                 trainable=True)
    
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

레이어의 `__call__` 메소드는 자동적으로 `build` 메소드를 가장 먼저 호출하여 수행합니다. 이제 게으르고(lazy) 사용하기 쉬운 레이어가 완성되었습니다.

In [9]:
linear_layer = Linear(32) # 인스턴스를 생성하는 단계에서, 호출될 입력 데이터에 대해 알지 못합니다.
y = linear_layer(x) # 레이어의 가중치는 해당 레이어가 호출되었을 때 가장 먼저 동적으로 생성됩니다.

### 재귀적으로 레이어 구성하기
만약, 레이어 인스턴스를 다른 레이어의 속성으로서 할당하게되면, 외부 레이어는 내부 레이어의 가중치를 트래킹하기 시작합니다.<br><br>
`__init__` 메소드 내에서 이러한 서브레이어를 생성하는 것이 좋습니다. (서브레이어들은 일반적으로 `build` 메소드를 가지고 있으므로 외부 레이어가 생성(build)될 때 해당 레이어들도 같이 생성됩니다.)

In [10]:
# Linear 클래스를 재사용한다고 가정해봅시다.
# `build` 메소드는 위에서 정의한대로 사용합니다.

class MLPBlock(layers.Layer):
    
    def __init__(self):
        super(MLPBlock, self).__init__()
        self.linear_1 = Linear(32)
        self.linear_2 = Linear(32)
        self.linear_3 = Linear(1)
        
    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)
    

mlp = MLPBlock()
y = mlp(tf.ones(shape=(3, 64)))
print('weights:', len(mlp.weights))
print('trainable weights:', len(mlp.trainable_weights))

weights: 6
trainable weights: 6


### 데이터가 모델을 통과하는 동안 생성되는 손실 값을 재귀적으로 수집하는 레이어
`call` 메소드를 작성할 때, 나중에 학습 루프를 작성할 때 사용할 손실 텐서를 만들 수 있습니다.<br>
이는 `self.add_loss(value)`를 호출해서 만들 수 있습니다.

In [11]:
# 활성화 정규화 손실 함수를 생성하는 레이어
class ActivityRegularizationLayer(layers.Layer):
    
    def __init__(self, rate=1e-2):
        super(ActivityRegularizationLayer, self).__init__()
        self.rate = rate
        
    def call(self, inputs):
        self.add_loss(self.rate * tf.reduce_sum(inputs))
        return inputs

이러한 손실 값들은(내부 레이어 어디서든 생성되는 것들을 포함하여) `layer.losses`를 통해 값을 확인할 수 있습니다.<br>
해당 값은 맨 앞 레이어의 `__call__`이 호출될 때마다 초기화 됩니다. 그래서 `layer.losses`는 마지막 학습에서 생성된 손실 값을 가지고 있게 됩니다.

In [12]:
class OuterLayer(layers.Layer):
    
    def __init__(self):
        super(OuterLayer, self).__init__()
        self.activity_reg = ActivityRegularizationLayer(1e-2)
        
    def call(self, inputs):
        return self.activity_reg(inputs)
    
layer = OuterLayer()
assert len(layer.losses) == 0 # 레이어가 아직 호출되지 않았기 때문에 손실 값이 존재하지 않습니다.
_ = layer(tf.zeros(1, 1))
assert len(layer.losses) == 1 # 레이어를 호출함으로써 손실 값 하나를 생성했습니다.

# `layer.losses` 는 각 __call__ 메소드의 시작에서 초기화됩니다.
_ = layer(tf.zeros(1, 1))
assert len(layer.losses) == 1 # 현재 손실 값은 마지막 호출에서 생성된 값 입니다.

추가적으로 `loss`는 내부 레이어의 가중치에서 생성된 손실 값도 포함하고 있습니다.

In [13]:
class OuterLayer(layers.Layer):
    
    def __init__(self):
        super(OuterLayer, self).__init__()
        self.dense = layers.Dense(32, kernel_regularizer = tf.keras.regularizers.l2(1e-3))
        
    def call(self, inputs):
        return self.dense(inputs)
    
    
layer = OuterLayer()
_ = layer(tf.zeros((1, 1)))

# 위의 'kernel_regularizaer'에서 생성된 값은 
# 1e-3 * sum(layer.dense.kernel ** 2)와 같습니다.
print(layer.losses)

[<tf.Tensor: shape=(), dtype=float32, numpy=0.0015565304>]


이러한 손실 값들은 학습 루프를 작성할 때, 다음과 같이 고려됩니다.

In [14]:
# # 옵티마이저를 생성 합니다.
# optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
# loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# # 데이터셋의 배치를 사용해 반복합니다.
# for x_batch_train, y_batch_train in train_dataset:
#     with tf.GradientTape() as tape:
#         logits = layer(x_batch_train) # 미니 배치의 로짓
#         loss_value = loss_fn(y_batch_train, logits) # 미니 배치의 손실 값
#         loss_value += sum(model.losses) # 순전파가 진행되는 동안 생성된 추가 로스 더하기
        
#     grads = tape.gradient(loss_Value, model.trainable_weights)
#     optimizer.apply_gradients(zip(grads, model.trainable_weights))

학습 루프에 대한 자세한 가이드는 두 번째 섹션의 `guide to training and evaluation`에서 다룹니다.<br><br>
### 선택적으로 레이어 직렬화 활성화하기
함수형 모델의 일부분으로써 맞춤형 레이어를 직렬화 하고자 한다면, 선택적으로 `get_config` 메소드를 구현할 수 있습니다.

In [15]:
class Linear(layers.Layer):
    
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units
        
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='random_normal',
                                 trainable=True)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
    def get_config(self):
        return {'units': self.units}
    

# 이제 해당 레이어가 가진 구성 정보로부터 레이어를 재생성 할 수 있습니다.
layer = Linear(64)
config = layer.get_config()
print(config)
new_layer = Linear.from_config(config)

{'units': 64}


부모 `Layer` 클래스의 `__init__` 메소드는 `name`이나 `dtype` 같은 몇개의 키워드 인자를 가지고 있습니다. <br>
이러한 인자들을 `__init__`에 정의된 부모 클래스에 전달하고 레이어의 구성 정보(config)에 포함시키는 것이 좋습니다.

In [16]:
class Linear(layers.Layer):
    
    def __init__(self, units=32, **kwargs):
        super(Linear, self).__init__(**kwargs)
        self.units = units
        
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                                 initializer='random_normal',
                                 trainable=True)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
    def get_config(self):
        config = super(Linear, self).get_config()
        config.update({'units': self.units})
        return config
    
layer = Linear(64)
config = layer.get_config()
print(config)
new_layer = Linear.from_config(config)

{'name': 'linear_8', 'trainable': True, 'dtype': 'float32', 'units': 64}


해당 레이어가 가진 구성 정보로부터 역직렬화를 할 때 더 많은 유연성이 필요하다면, `from_config` 메소드를 오버라이드 할 수 있습니다.<br>
`from_config`의 기본 구현은 다음과 같습니다.

In [17]:
def from_config(cls, config):
    return cls(**config)

직렬화와 저장에 대해 더 알아보기 위해서는 [Guide to Savinng and Serializing Models](https://www.tensorflow.org/guide/keras/save_and_serialize) 를 살펴보세요.<br><br>
### call 메소드의 training 인자가 가지는 특권
몇몇 레이어, 특히 `BatchNormalization` 레이어와 `Dropout` 레이어는 훈련과 추론 단계에서 서로 다르게 작동합니다.<br>
이러한 레이어를 위한 표준으로 `call` 메소드 내에서 `training`(boolean) 인자를 사용할 수 있습니다.<br><br>
`training` 인자를 사용하여, 레이어 내부에서 내장된 학습 및 검증 루프가 학습이나 추론단계에서 적절하게 작동할 수 있도록 할 수 있습니다.

In [18]:
class CustomDropout(layers.Layer):
    
    def __init__(self, rate, **kwargs):
        super(CostomDroupout, self).__init__(**kwargs)
        self.rate = rate
        
    def call(self, inputs, training=None):
        if training:
            return tf.nn.dropout(inputs, rate=self.rate)
        return inputs

## 모델 작성하기
### Model 클래스
일반적으로, 내부 연산 블록을 정의하기 위해 `Layer` 클래스를 사용하게 됩니다. 그리고 나중에 사용할 객체로써 `Model` 클래스를 사용하여 외부 모델을 정의합니다. <br><br>
레즈넷50 모델 인스턴스는 `Layer` 클래스의 서브클래스로 이루어진 여러 레즈넷 블록을 가지고 있습니다. 전체 레즈넷 네트워크를 하나의 `Model` 클래스가 감싸고 있습니다.<br><br>
`Model` 클래스는 `Layer` 클래스와 같은 API를 가지고 있으며, 다음과 같은 차이점이 있습니다.<br>
- 학습, 검증, 예측 루프에 사용되는 내장 함수를 가지고 있습니다. (`model.fit()`, `model.evaluate()`, `model.predict()`)
- `model.layers` 속성을 통해 내부 레이어의 리스트에 접근할 수 있습니다.
- 저장 및 직렬화 API를 가지고 있습니다.

효과적으로, "Layer" 클래스는 우리가 'layer'라고 부르는 것(convolution layer, recurrent layer) 또는 'block'이라 부르는 것('Resnet block' or 'Inception block')에 대응합니다.<br><br>
한편, "Model" 클래스는 우리가 'model'이라고 부르는 것(deep learning model)이나 'network'라고 부르는 것(deep neural network)에 대응합니다.<br><br>
작은 레즈넷 모델 예제를 살펴보고, 모델을 생성하기 위해 `Model` 클래스를 사용하겠습니다. 그리고 `fit()` 함수를 사용해 학습을 진행한 뒤에 `save_weights` 함수로 모델을 저장하겠습니다.

In [19]:
# class ResNet(tf.keras.Model):
    
#     def __init__(self):
#         super(Resnet, self).__init__()
#         self.block_1 = ResNetBlock()
#         self.block_2 = ResNetBlock()
#         self.global_pool = layers.GlobalAveragePooling2D()
#         self.classifier = Desne(num_classes)
        
#     def call(self, inputs):
#         x = self.block_1(inputs)
#         x = self.block_2(x)
#         x = self.global_pool(x)
#         return self.classifier(x)
    
# resnet = ResNet()
# dataset = ...
# resnet.fit(dataset, epochs=10)
# resnet.save_weights(filepath)

### 하나로 합치기: 엔드 투 엔드 예제
지금까지 배운 내용은 다음과 같습니다.
- `Layer` 클래스는 `__init__` 또는 `build`로 생성된 내부 상태와 `call` 함수 내부의 연산을 캡슐화 한다.
- Layer 클래스는 새로운 거대한 연산을 위한 블록을 생성하기 위해 반복적으로 중첩됩니다.
- Layer 클래스는 손실 함수(일반적으로 정규화 손실 함수)를 생성하고 추적할 수 있습니다.
- 학습하고자 하는 외부 컨테이너는 `Model` 클래스이며, `Model` 클래스는 `Layer` 클래스와 비슷하지만 학습 및 직렬화 유틸리티가 추가되었습니다.<br><br>
이러한 것들을 하나로 합쳐 엔드 투 엔드 예제를 만들어보겠습니다. MNIST 숫자 이미지를 생성하는 Variational AutoEncoder를 구현해보겠습니다. <br><br>
VAE는 `Model` 클래스를 서브클래싱하고 `Layer` 클래스의 서브클래스로 이루어진 레이어들의 중첩 구조로 구성합니다. 그리고 KL Divergence라 불리는 정규화 손실 함수를 사용합니다.

In [20]:
class Sampling(layers.Layer):
    # (z_mean, z_log_var) 는 z로 부터 샘플링되며, 벡터는 숫자를 인코딩한 결과입니다.
    
    def call(sefl, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 + z_log_var) * epsilon
    
    
class Encoder(layers.Layer):
    # MNIST 숫자를 (z_mean, z_log_var, z)이 세가지 값으로 매핑합니다.
    
    def __init__(self,
                 latent_dim=32,
                 intermediate_dim=64,
                 name='encoder',
                 **kwargs):
        super(Encoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation='relu')
        self.dense_mean = layers.Dense(latent_dim)
        self.dense_log_var = layers.Dense(latent_dim)
        self.sampling = Sampling()
    
    def call(self, inputs):
        x = self.dense_proj(inputs)
        z_mean = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.sampling((z_mean, z_log_var))
        return z_mean, z_log_var, z
    
class Decoder(layers.Layer):
    # z를 변환하여 인코딩된 숫자 벡터를 읽을 수 있는 숫자로 되돌립니다.
    
    def __init__(self,
                 original_dim,
                 intermediate_dim=64,
                 name='decoder',
                 **kwargs):
        super(Decoder, self).__init__(name=name, **kwargs)
        self.dense_proj = layers.Dense(intermediate_dim, activation='relu')
        self.dense_output = layers.Dense(original_dim, activation='sigmoid')
        
    def call(self, inputs):
        x = self.dense_proj(inputs)
        return self.dense_output(x)
    

class VariationalAutoEncoder(tf.keras.Model):
    # 인코더와 디코더를 연결하여 엔드 투 엔드 학습 모델을 구성합니다.
    
    def __init__(self,
                 original_dim,
                 intermediate_dim=64,
                 latent_dim=32,
                 name='autoencoder',
                 **kwargs):
        super(VariationalAutoEncoder, self).__init__(name=name, **kwargs)
        self.original_dim = original_dim,
        self.encoder = Encoder(latent_dim=latent_dim,
                              intermediate_dim=intermediate_dim)
        self.decoder = Decoder(original_dim, intermediate_dim=intermediate_dim)
        
    def call(self, inputs):
        z_mean, z_log_var, z = self.encoder(inputs)
        reconstructed = self.decoder(z)
        # KL divergence 손실 함수 추가
        kl_loss = - 0.5 * tf.reduce_mean(
                z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
        self.add_loss(kl_loss)
        return reconstructed

In [21]:
original_dim = 784
vae = VariationalAutoEncoder(original_dim, 64, 32)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
mse_loss_fn = tf.keras.losses.MeanSquaredError()

loss_metric = tf.keras.metrics.Mean()

(x_train, _), _ = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(60000, 784).astype('float32') / 255

train_dataset = tf.data.Dataset.from_tensor_slices(x_train)
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(64)

epochs = 3

# 에폭 수 만큼 반복
for epoch in range(epochs):
    print('Start of epoch %d' % (epoch,))
    
    # 데이터 셋의 배치 크기 만큼 데이터를 반복 학습
    for step, x_batch_train in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            reconstructed = vae(x_batch_train)
            # Reconstruction loss 계산
            loss = mse_loss_fn(x_batch_train, reconstructed)
            loss += sum(vae.losses) # KLD 손실을 더함
            
        grads = tape.gradient(loss, vae.trainable_weights)
        optimizer.apply_gradients(zip(grads, vae.trainable_weights))
        
        loss_metric(loss)
        
        if step % 100 == 0:
            print('step %s: mean loss = %s' % (step, loss_metric.result()))

Start of epoch 0
step 0: mean loss = tf.Tensor(0.37233296, shape=(), dtype=float32)
step 100: mean loss = tf.Tensor(0.11438663, shape=(), dtype=float32)
step 200: mean loss = tf.Tensor(0.09296365, shape=(), dtype=float32)
step 300: mean loss = tf.Tensor(0.085019186, shape=(), dtype=float32)
step 400: mean loss = tf.Tensor(0.081082925, shape=(), dtype=float32)
step 500: mean loss = tf.Tensor(0.07838162, shape=(), dtype=float32)
step 600: mean loss = tf.Tensor(0.07667232, shape=(), dtype=float32)
step 700: mean loss = tf.Tensor(0.07537877, shape=(), dtype=float32)
step 800: mean loss = tf.Tensor(0.07444117, shape=(), dtype=float32)
step 900: mean loss = tf.Tensor(0.0736006, shape=(), dtype=float32)
Start of epoch 1
step 0: mean loss = tf.Tensor(0.0733748, shape=(), dtype=float32)
step 100: mean loss = tf.Tensor(0.072845384, shape=(), dtype=float32)
step 200: mean loss = tf.Tensor(0.07246542, shape=(), dtype=float32)
step 300: mean loss = tf.Tensor(0.07206686, shape=(), dtype=float32)
ste

VAE 모델은 `Model` 클래스를 서브클래싱한 것이므로, 내장된 학습 루프를 사용합니다.<br>
다음과 같이 수행됩니다.

In [22]:
vae = VariationalAutoEncoder(784, 64, 32)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

vae.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
vae.fit(x_train, x_train, epochs=3, batch_size=64)

Train on 60000 samples
Epoch 1/3
60000/60000 [==============================] - 2s 31us/sample - loss: 0.0733
Epoch 2/3
60000/60000 [==============================] - 2s 26us/sample - loss: 0.0677
Epoch 3/3
60000/60000 [==============================] - 2s 30us/sample - loss: 0.0676


## 객체 지향 개발을 넘어서: 함수형 API
이번 예제가 너무 객체 지향적이지는 않으셨나요? 함수형 API를 사용해서도 모델을 작성하실 수 있습니다.<br>
중요한 점은, 하나의 스타일을 선택하는 것이 다른 스타일을 사용하여 작성된 구성요소를 방해하지 않는다는 점입니다. 원한다면 두 스타일을 적재적소에 섞을 수 있습니다. <br><br>
예제로, 위에서 작성했던 `Sampling` 레이어를 재사용하여 함수형 API 예제를 살펴보겠습니다.

In [23]:
original_dim = 784
intermediate_dim = 64
latent_dim = 32

# 인코더 모델 정의
original_inputs = tf.keras.Input(shape=(original_dim,), name='encoder_input')
x = layers.Dense(intermediate_dim, activation='relu',)(original_inputs)
z_mean = layers.Dense(latent_dim, name='z_mean')(x)
z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)
z = Sampling()((z_mean, z_log_var))
encoder = tf.keras.Model(inputs=original_inputs, outputs=z, name='encoder')

# 디코더 모델 정의
latent_inputs = tf.keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
outputs = layers.Dense(original_dim, activation='sigmoid')(x)
decoder = tf.keras.Model(inputs=latent_inputs, outputs=outputs, name='decoder')

# VAE 모델 정의
outputs = decoder(z)
vae = tf.keras.Model(inputs=original_inputs, outputs=outputs, name='vae')

# KL Divergence 손실 함수 추가
kl_loss = - 0.5 * tf.reduce_mean(
        z_log_var - tf.square(z_mean) - tf.exp(z_log_var) + 1)
vae.add_loss(kl_loss)

# 학습
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
vae.compile(optimizer, loss=tf.keras.losses.MeanSquaredError())
vae.fit(x_train, x_train, epochs=3, batch_size=64)

Train on 60000 samples
Epoch 1/3
60000/60000 [==============================] - 2s 35us/sample - loss: 0.0731
Epoch 2/3
60000/60000 [==============================] - 2s 30us/sample - loss: 0.0677
Epoch 3/3
60000/60000 [==============================] - 2s 29us/sample - loss: 0.0676
